# Imports

In [3]:
import torch
from torch import nn
import torch.nn.functional as F

# Tokenization

## Character level encoding

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

## BPE

In [ ]:
# BPE algorithm

## Prepare splits

In [ ]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split, block_size, batch_size, device='cpu'):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model, eval_iters=10):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Attention

In [ ]:
class CausalHead(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.k = nn.Linear(config.n_embd, config.head_size, bias=False) #(B, T, C) @ (C, H) -> (B, T, H)
        self.q = nn.Linear(config.n_embd, config.head_size, bias=False)
        self.v = nn.Linear(config.n_embd, config.head_size, bias=False)
        self.register_buffer("cmask", torch.tril(torch.ones([config.ctx_len, config.ctx_len])))#  define causal mask
        self.head_size = config.head_size

    def forward(self, x):
        _, T, _ = x.shape

        K = self.k(x)
        Q = self.q(x)
        V = self.v(x)

        a = Q @ torch.transpose(K, -2, -1) * self.head_size**(-0.5) #(B, T, H) @ (B, H, T) -> (B, T, T)
        torch.masked_fill(a, self.cmask[:T,:T]==0, float('-inf'))  #  apply mask
        a = F.softmax(a, dim=-1)
        return a @ V #(B, T, T) @ (B, T, H) -> (B, T, H)

In [ ]:
class MultiHeadCausal(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.mha = nn.ModuleList([CausalHead() for _ in range(config.num_heads)])
        self.lin = nn.Linear(config.n_embd, config.n_embd)
        self.dp = nn.Dropout(config.dropout_p)

    def forward(self, x):
        a = torch.cat([h(x) for h in self.mha], dim=-1)
        a = self.lin(a)
        a = self.dp(a)
        return a


## Define decoder block

In [6]:
class FF(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.l1 = nn.Linear(config.n_embd, 4*config.n_embd)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(4*config.n_embd, config.n_embd)
        self.dp = nn.Dropout(config.dropout_p)

    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.dp(x)
        return x

class DecoderBlock(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.mha = MultiHeadCausal()
        self.FF = FF()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)

    def forward(self, x):
        x = x + self.mha(self.ln1(x))
        x = x + self.FF(self.ln2(x))
        return x

# Define model

In [ ]:
class Decoder(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.blocks = nn.Sequential(*[DecoderBlock() for _ in range(n_layer)])
        self.ln = nn.LayerNorm(config.n_embd)
        self.lin = nn.Linear(config.n_embd, vocab_size)
        self.wte = nn.Embedding(vocab_size, config.n_embd)
        self.wpe = nn.Embedding(config.ctx_len, config.n_embd) #many possible choices for this, there are tradeoffs
        self.L = nn.CrossEntropyLoss()
        self.ctx_len = config.ctx_len


    def forward(self, x, targets=[]):
        #x is not (B, T, C) it's a sequence of tokenized inputs
        #it is (B, T)
        B, T = x.shape

        x_tok = self.wte(x)
        x_pos = self.wpe(torch.arange(T))
        x = x_pos + x_tok #(B, T, C)

        x = self.blocks(x)
        x = self.ln(x)
        logits = self.lin(x) #(B, T, vocab_size)

        if len(targets)==0:
            loss = None
        else:
            #compute xentropy loss...
            B, T, C = logits.shape
            targets = targets.view(B*T)
            logits = logits.view(B*T, C)
            loss = self.L(logits, targets)


        return logits, loss

    def generate(self, idx, max_len=128):

        #generate output
        #idx is (B, T)
        for _ in range(max_len):
            idx_cond = idx[:,-self.ctx_len:]
            logits, _ = self(idx_cond)
            logits = logits[:,-1,:] #  get last token

            prob = F.softmax(logits, dim=-1)

            next_token = torch.multinomial(prob, num_samples=1) #  greedy sample next token; many possibilities here (e.g., nucleus, probabilistic, etc.)

            idx = torch.cat((idx, next_token), dim=1)

        return idx

# Optimizations

## Embeddings

## MQA and GQA and SWA

## Normalization and activation functions

# Train the model